In [1]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    BertTokenizerFast,
    BertForSequenceClassification,
    BertTokenizer,
    AutoTokenizer
)
from huggingface_hub import HfFolder, notebook_login
import os

os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from datasets import Dataset
import pandas as pd

full_dataset = pd.read_csv("round5.csv", nrows=20000)

In [3]:
# model_id = "roberta-base"
model_id = "bert-base-uncased"

In [4]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
train_dataset, test_dataset = train_test_split(full_dataset, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [5]:
dataset = Dataset.from_pandas(full_dataset)

In [6]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Assuming 'predicted' is the column with class names
unique_classes = full_dataset['predicted'].unique()
class_to_index = {label: idx for idx, label in enumerate(unique_classes)}

# Add this to your tokenize function to use this mapping
def tokenize(batch):
    tokenized_inputs = tokenizer(batch['concatenated_text'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs['labels'] = [class_to_index[label] for label in batch['labels']]
    return tokenized_inputs



# Tokenize the datasets
train_dataset = train_dataset.rename_column('predicted', 'labels')
test_dataset = test_dataset.rename_column('predicted', 'labels')
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# You might need to set format for PyTorch

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\mihae\anaconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mihae\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [7]:
full_dataset

,concatenated_text,predicted
0,White Horse;Tile Manufacturing | European Aest...,Building Material and Garden Equipment and Sup...
1,Wealth Solution Partners;Super and SMSF Servic...,"Funds, Trusts, and Other Financial Vehicles"
2,PMG;Fire and Water Cleanup Services | Mold Rem...,Waste Management and Remediation Services
3,TMP Capital PLLC;Licensed in AL & FL | 203K Lo...,Credit Intermediation and Related Activities
4,Genertek Power;Industrial and Commercial Energ...,Utilities
...,...,...
19995,Pros DIY;Kitchen Sink Drains | DIY Project Tut...,Repair and Maintenance
19996,Dajo Hair Salon Studio;Personal Care Services ...,Personal and Laundry Services
19997,Big Accessories;Customization Services for Bag...,"Sporting Goods, Hobby, Musical Instrument, Boo..."
19998,Canadian Urban;Real Estate Syndication | Real ...,Real Estate


In [8]:
repository_id = "output"

!nvidia-smi

# Model
model = BertForSequenceClassification.from_pretrained(model_id)

model = model.to(device)

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from datasets import load_metric
import numpy as np
from transformers import Trainer, TrainingArguments

# data collator is used to collate the data into batches that can be fed to the model during training and evaluation
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

# define metrics and metrics function
f1_metric = load_metric("f1")
accuracy_metric = load_metric( "accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="micro")
    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"],
    }

training_args = TrainingArguments(
    output_dir='output_dir',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Make sure this function is appropriate for your classification
    tokenizer = tokenizer,
#     data_collator=data_collator
)

trainer.train()

C:\Users\mihae\AppData\Local\Temp\ipykernel_28532\3675541049.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric = load_metric("f1")
C:\Users\mihae\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\mihae\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You 

In [ ]:
for batch in train_dataset:
    print(batch['input_ids'].shape, batch['attention_mask'].shape, batch['labels'])
    break  # Just print the first batch to check

In [ ]:
trainer.train()